In [1]:
import os
import parse
import pickle

import torch
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
import dionysus as dion
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import svm
import sklearn
import networkx as nx
from grakel import GraphKernel
import grakel
import seaborn as sns

from pt_activation.models.simple_mnist_sigmoid import CFF

%load_ext autoreload
%autoreload 2

In [2]:
def get_adv_info(filename):
    format_string = 'true-{}_adv-{}_sample-{}.npy'
    parsed = parse.parse(format_string, filename)
    return {'true class':int(parsed[0]), 'adv class':int(parsed[1]), 'sample':int(parsed[2])}

def read_adversaries(loc):
    ret = []
    for f in os.listdir(loc):
        if os.path.isfile(os.path.join(loc,f)) and f.find('.npy') != -1:
            adv = np.load(os.path.join(loc, f))
            info = get_adv_info(f)
            info['adversary'] = adv
            ret.append(info)
    return ret
    

In [3]:
adv_directory_loc = '/home/tgebhart/projects/pt_activation/logdir/adversaries/additive_gaussian_noise/cff_3-filters_8-kernel_size_50-fc1_sigmoid-activation.pt/90'
adversaries = read_adversaries(adv_directory_loc)
adversaries = sorted(adversaries,  key=lambda k: k['sample'])

def create_filtrations(model, batch_size, up_to):
    device = torch.device("cpu")
    kwargs = {'num_workers': 1, 'pin_memory': True}
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, download=True, transform=transforms.Compose([
                           transforms.ToTensor(),
                       ])), batch_size=batch_size, shuffle=False, **kwargs)

    model.eval()
    test_loss = 0
    correct = 0
    t = 0
    res_df = []
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output, hiddens = model(data, hiddens=True)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
            for s in range(data.shape[0]):
                row = {'loss':output.cpu().numpy()[s][0], 'class':target.cpu().numpy()[s], 'prediction':pred.cpu().numpy()[s][0]}
                res_df.append(row)

            t += batch_size
            if t >= up_to:
                break

    return pd.DataFrame(res_df)


def create_adversary_filtrations(model, batch_size, up_to, adversaries):
    device = torch.device("cpu")
    adv_images = torch.tensor(np.array([a['adversary'] for a in adversaries]))
    adv_labels = torch.tensor(np.array([a['true class'] for a in adversaries]))
    adv_samples = [a['sample'] for a in adversaries]
    
    print(adv_images.shape, adv_labels.shape)
    
    advs = torch.utils.data.TensorDataset(adv_images, adv_labels)
    test_loader = torch.utils.data.DataLoader(advs, batch_size=batch_size, shuffle=False)
    
    model.eval()
    test_loss = 0
    correct = 0
    t = 0
    res_df = []
    with torch.no_grad():
        
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output, hiddens = model(data, hiddens=True)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
            for s in range(data.shape[0]):
                row = {'loss':output.cpu().numpy()[s][0], 'class':target.cpu().numpy()[s], 'prediction':pred.cpu().numpy()[s][0], 'sample':adv_samples[s+t]}
                res_df.append(row)

            t += (batch_size)
            if t >= up_to:
                break

    return pd.DataFrame(res_df)

In [4]:
def create_filtrations(model, batch_size, up_to):
    device = torch.device("cpu")
    kwargs = {'num_workers': 1, 'pin_memory': True}
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, download=True, transform=transforms.Compose([
                           transforms.ToTensor(),
                       ])), batch_size=batch_size, shuffle=False, **kwargs)

    model.eval()
    test_loss = 0
    correct = 0
    t = 0
    res_df = []
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output, hiddens = model(data, hiddens=True)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
            for s in range(data.shape[0]):
                # check if this makes sense
                this_hiddens = [hiddens[0][s], hiddens[1][s], hiddens[2][s]]
                print('Filtration: {}'.format(s+t))
                f = model.compute_dynamic_filtration(data[s,0], this_hiddens)
                #
                row = {'filtration':f, 'loss':output.cpu().numpy()[s][0], 'class':target.cpu().numpy()[s], 'prediction':pred.cpu().numpy()[s][0]}
                res_df.append(row)

            t += batch_size
            if t >= up_to:
                break

    return pd.DataFrame(res_df)


def create_adversary_filtrations(model, batch_size, up_to, adversaries):
    device = torch.device("cpu")
    adv_images = torch.tensor(np.array([a['adversary'] for a in adversaries]))
    adv_labels = torch.tensor(np.array([a['true class'] for a in adversaries]))
    
    print(adv_images.shape, adv_labels.shape)
    
    advs = torch.utils.data.TensorDataset(adv_images, adv_labels)
    test_loader = torch.utils.data.DataLoader(advs, batch_size=batch_size, shuffle=False)
    
    model.eval()
    test_loss = 0
    correct = 0
    t = 0
    res_df = []
    with torch.no_grad():
        
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output, hiddens = model(data, hiddens=True)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
            for s in range(data.shape[0]):
                # check if this makes sense
                this_hiddens = [hiddens[0][s], hiddens[1][s], hiddens[2][s]]
                print('Filtration: {}'.format(s+t))
                f = model.compute_dynamic_filtration(data[s,0], this_hiddens)
                #
                row = {'filtration':f, 'loss':output.cpu().numpy()[s][0], 'class':target.cpu().numpy()[s], 'prediction':pred.cpu().numpy()[s][0]}
                res_df.append(row)

            t += (batch_size)
            if t >= up_to:
                break

    return pd.DataFrame(res_df)

In [5]:
model_location = '/home/tgebhart/projects/pt_activation/logdir/models/cff_3-filters_8-kernel_size_50-fc1_sigmoid-activation.pt'
model = CFF()
model.load_state_dict(torch.load(model_location))

In [6]:
res_df = create_filtrations(model, 10, 10)

Filtration: 0
h1_id_start 784
h2_id_start 2107
h3_id_start 2157
filtration size 87973
Sorting filtration...
Filtration: 1
h1_id_start 784
h2_id_start 2107
h3_id_start 2157
filtration size 95357
Sorting filtration...
Filtration: 2
h1_id_start 784
h2_id_start 2107
h3_id_start 2157
filtration size 79401
Sorting filtration...
Filtration: 3
h1_id_start 784
h2_id_start 2107
h3_id_start 2157
filtration size 102834
Sorting filtration...
Filtration: 4
h1_id_start 784
h2_id_start 2107
h3_id_start 2157
filtration size 90089
Sorting filtration...
Filtration: 5
h1_id_start 784
h2_id_start 2107
h3_id_start 2157
filtration size 82683
Sorting filtration...
Filtration: 6
h1_id_start 784
h2_id_start 2107
h3_id_start 2157
filtration size 91718
Sorting filtration...
Filtration: 7
h1_id_start 784
h2_id_start 2107
h3_id_start 2157
filtration size 91370
Sorting filtration...
Filtration: 8
h1_id_start 784
h2_id_start 2107
h3_id_start 2157
filtration size 98219
Sorting filtration...
Filtration: 9
h1_id_start 7

In [ ]:
# adv_df = create_adversary_filtrations(model, 50, 1000, adversaries)

In [ ]:
res_df = create_filtrations(model, 10, 10)

In [7]:
def create_sample_graphs(df):
    sample_graphs = []
    for s in range(df.shape[0]):
        print(s)
        subgraphs = {}
        f = df['filtration'].iloc[s]
        m = dion.homology_persistence(f)
        dgms = dion.init_diagrams(m,f)
        for i,c in enumerate(m):
            if len(c) == 2:
                if f[c[0].index][0] in subgraphs:
                    subgraphs[f[c[0].index][0]].add_edge(f[c[0].index][0],f[c[1].index][0],weight=f[i].data)
                else:
                    eaten = False
                    for k, v in subgraphs.items():
                        if v.has_node(f[c[0].index][0]):
                            v.add_edge(f[c[0].index][0], f[c[1].index][0], weight=f[i].data)
                            eaten = True
                            break
                    if not eaten:
                        g = nx.Graph()
                        g.add_edge(f[c[0].index][0], f[c[1].index][0], weight=f[i].data)
                        subgraphs[f[c[0].index][0]] = g

        sample_graphs.append(subgraphs)
    return sample_graphs

In [8]:
sample_graphs = create_sample_graphs(res_df)

0
1
2
3
4
5
6
7
8
9


In [ ]:
# adv_sample_graphs = create_sample_graphs(adv_df)
# with open(os.path.join(adv_directory_loc, 'adv_samples.pkl'), 'wb') as f:
#     pickle.dump(adv_sample_graphs, f)
# sample_graphs = create_sample_graphs(res_df)
# with open(os.path.join(adv_directory_loc, 'samples.pkl'), 'wb') as f:
#     pickle.dump(sample_graphs, f)

# sample_graphs = pickle.load( open(os.path.join('/home/tgebhart/projects/pt_activation/logdir/adversaries/lbfgsm/cff_3-filters_8-kernel_size_50-fc1_sigmoid-activation.pt/90', 'samples.pkl'), "rb") )
# adv_sample_graphs = pickle.load( open(os.path.join(adv_directory_loc, 'adv_samples.pkl'), "rb") )

In [ ]:
# options = {
#     'node_color': 'red',
#     'node_size': 2,
#     'width': 3,
#     'with_labels':True}
# nx.draw_random(subgraphs[243], **options)

In [ ]:
adv_df.head()

In [ ]:
res_df.head()

In [ ]:
incorrects = res_df[res_df['class'] != res_df['prediction']]
corrects = res_df[res_df['class'] == res_df['prediction']]

In [ ]:
incorrects.shape

In [ ]:
incorrects.head(20)

In [ ]:
correct_list = list(corrects.index)

In [ ]:
adv_correct_list = list(adv_df[~np.isin(adv_df['sample'], list(incorrects.index))].index)

In [ ]:
sample_graphs = [sample_graphs[i] for i in correct_list]
adv_sample_graphs = [adv_sample_graphs[i] for i in adv_correct_list]

In [ ]:
COLORS = ['#12355b', '#ff6978']
EDGE_COLOR = '#272d2d'
PLT_LABELS = ['Unaltered', 'Adversarial']
# COLORS = ['#bfdce7', '#ff6978']

In [ ]:
sgl = np.zeros(len(sample_graphs))
for i in range(len(sample_graphs)):
    sgl[i] = len(sample_graphs[i])

adv_sgl = np.zeros(len(adv_sample_graphs))
for i in range(len(adv_sample_graphs)):
    adv_sgl[i] = len(adv_sample_graphs[i])


In [ ]:
plt.hist([sgl, adv_sgl], bins='auto', color=COLORS, label=PLT_LABELS, edgecolor=EDGE_COLOR)
plt.legend()
plt.title('Distribution of Generator Count')
plt.xlabel('Number of Generators')

In [ ]:
take = 10
edges = set()
for i in range(len(sample_graphs)):
    for k in list(sample_graphs[i].keys())[:take]:
        for x in sample_graphs[i][k].edges(data=True):
            edge_name = str(x[0])+'-'+str(x[1])
            edges.add(edge_name)

In [ ]:
for i in range(len(adv_sample_graphs)):
    for k in list(adv_sample_graphs[i].keys())[:take]:
        for x in adv_sample_graphs[i][k].edges(data=True):
            edge_name = str(x[0])+'-'+str(x[1])
            edges.add(edge_name)

In [ ]:
edf = pd.DataFrame(np.zeros((len(sample_graphs)+len(adv_sample_graphs),len(edges))), columns=list(edges))
for i in range(len(sample_graphs)):
    print('Sample: {}/{}'.format(i,len(sample_graphs)))
    for k in list(sample_graphs[i].keys())[:take]:
        for x in sample_graphs[i][k].edges(data=True):
            edge_name = str(x[0])+'-'+str(x[1])
            edf.iloc[i][edge_name] = x[2]['weight']
            
for i in range(len(adv_sample_graphs)):
    print('Sample: {}/{}'.format(i,len(adv_sample_graphs)))
    for k in list(adv_sample_graphs[i].keys())[:take]:
        for x in adv_sample_graphs[i][k].edges(data=True):
            edge_name = str(x[0])+'-'+str(x[1])
            edf.iloc[i+len(sample_graphs)][edge_name] = x[2]['weight']

            

In [ ]:
min_max_scaler = sklearn.preprocessing.MinMaxScaler()
scaled = min_max_scaler.fit_transform(np.concatenate((sgl, adv_sgl), axis=0).reshape(-1,1))
edf['graph counts'] = pd.Series(scaled.T.reshape(-1), index=edf.index)

In [ ]:
edf.tail()

In [ ]:
X = edf.values[:len(sample_graphs)]
y = res_df['class'].values

In [ ]:
clf = svm.SVC(gamma='scale', decision_function_shape='ovo').fit(X,y)

In [ ]:
X_adv = edf.values[len(sample_graphs):]
adv_preds = clf.predict(X_adv)

In [ ]:
sklearn.metrics.accuracy_score(adv_df['class'], adv_preds)

In [ ]:
adv_preds[:10]

In [ ]:
adv_df.head(10)

In [ ]:
sklearn.metrics.accuracy_score(adv_df['prediction'], adv_preds)

In [ ]:
clf2 = svm.SVC(gamma='scale', decision_function_shape='ovo').fit(X[:-100],y[:-100])

In [ ]:
sklearn.metrics.accuracy_score(y[-100:], clf2.predict(X[-100:]))

In [ ]:
all_gois = []
for i in range(len(sample_graphs)):
    print(i)
    a = [sample_graphs[i][k] for k in sample_graphs[i].keys()]
    all_gois.append(nx.compose_all(a))

In [ ]:
adv_all_gois = []
for i in range(len(adv_sample_graphs)):
    print(i)
    a = [adv_sample_graphs[i][k] for k in adv_sample_graphs[i].keys()]
    adv_all_gois.append(nx.compose_all(a))

In [ ]:
eigs = []
for i in range(len(all_gois)):
    print('normal ', i)
    eigs.append(nx.linalg.laplacian_spectrum(all_gois[i]))

adv_eigs = []
for i in range(len(adv_all_gois)):
    print('adv ', i)
    adv_eigs.append(nx.linalg.laplacian_spectrum(adv_all_gois[i]))

In [ ]:
num_top_eigs = 800
top_eigs = np.zeros((len(eigs),num_top_eigs))
for i in range(len(eigs)):
#     top_eigs[i] = np.sort(eigs[i])[::-1][:num_top_eigs] # descending
    top_eigs[i] = np.sort(eigs[i])[:num_top_eigs] # ascending
    
adv_top_eigs = np.zeros((len(adv_eigs),num_top_eigs))
for i in range(len(adv_eigs)):
#     adv_top_eigs[i] = np.sort(adv_eigs[i])[::-1][:num_top_eigs] # descending
    adv_top_eigs[i] = np.sort(adv_eigs[i])[:num_top_eigs] # ascending

In [ ]:
plt.hist([np.amax(top_eigs, axis=1),np.amax(adv_top_eigs, axis=1)], bins='auto', color=COLORS, label=PLT_LABELS, edgecolor=EDGE_COLOR)
plt.legend()
plt.title('Distribution of Maximum Eigenvalue of Spectrum')

In [ ]:
plt.hist([np.mean(top_eigs, axis=1),np.mean(adv_top_eigs, axis=1)],bins='auto', color=COLORS, label=PLT_LABELS, edgecolor=EDGE_COLOR)
plt.legend()
plt.title('Distribution of Mean Eigenvalue of Spectrum')

In [ ]:
plt.hist([np.median(top_eigs, axis=1),np.median(adv_top_eigs, axis=1)], range=[0,1.4], bins='auto', color=COLORS, label=PLT_LABELS, edgecolor=EDGE_COLOR)
plt.legend()
plt.title('Distribution of Median Eigenvalue of Spectrum')

In [ ]:
plt.hist([np.amin(top_eigs, axis=1),np.amin(adv_top_eigs, axis=1)], bins='auto', color=COLORS, label=PLT_LABELS, edgecolor=EDGE_COLOR)
plt.legend()
plt.title('Distribution of Minimum Eigenvalue of Spectrum')

In [ ]:
plt.hist([top_eigs[:,1], adv_top_eigs[:,1]], range=[0,0.006], bins='auto',color=COLORS, label=PLT_LABELS, edgecolor=EDGE_COLOR)
plt.legend()
plt.title('Distribution of Algebraic Connectivity')
plt.xlabel('Algebraic Connectivity')


In [ ]:
mean_eigs = np.zeros((len(eigs),3))
for i in range(len(eigs)):
    mean_eigs[i] = [eigs[i].mean(), eigs[i].shape[0], np.median(eigs[i])]
    
adv_mean_eigs = np.zeros((len(adv_eigs),3))
for i in range(len(adv_eigs)):
    adv_mean_eigs[i] = [adv_eigs[i].mean(), adv_eigs[i].shape[0], np.median(adv_eigs[i])]

In [ ]:
plt.hist([mean_eigs[:,0], adv_mean_eigs[:,0]], range=[2, 7], bins='auto', color=COLORS, label=PLT_LABELS, edgecolor=EDGE_COLOR)
plt.legend()
plt.title('Distribution of Mean Eigenvalues of Spectrum')

In [ ]:
plt.hist([mean_eigs[:,2], adv_mean_eigs[:,2]], range=[0.5,4], bins='auto', color=COLORS, label=PLT_LABELS, edgecolor=EDGE_COLOR)
plt.legend()
plt.title('Distribution of Median Eigenvalues of Spectrum')

In [ ]:
plt.hist([mean_eigs[:,1], adv_mean_eigs[:,1]], range=[1450,1950], bins='auto', color=COLORS, label=PLT_LABELS)
plt.legend()
plt.title('Distribution of Spectrum Size')

In [ ]:
np.where(adv_mean_eigs[:,1] < 1600)[0]

In [ ]:
some_num = 10
some_gois = []
for i in range(len(sample_graphs)):
    print(i)
    sgik = list(sample_graphs[i].keys())
    a = [sample_graphs[i][k] for k in sgik[len(sgik)-some_num:]]
    some_gois.append(nx.compose_all(a))
    
adv_some_gois = []
for i in range(len(adv_sample_graphs)):
    print(i)
    sgik = list(adv_sample_graphs[i].keys())
    a = [adv_sample_graphs[i][k] for k in sgik[len(sgik)-some_num:]]
    adv_some_gois.append(nx.compose_all(a))
    
some_eigs = []
for i in range(len(some_gois)):
    print('normal ', i)
    some_eigs.append(nx.linalg.laplacian_spectrum(some_gois[i]))

adv_some_eigs = []
for i in range(len(adv_some_gois)):
    print('adv ', i)
    adv_some_eigs.append(nx.linalg.laplacian_spectrum(adv_some_gois[i]))

In [ ]:
some_mean_eigs = np.zeros((len(some_eigs),5))
for i in range(len(some_eigs)):
    some_mean_eigs[i] = [some_eigs[i].mean(), some_eigs[i].shape[0], np.median(some_eigs[i]), some_eigs[i].min(), some_eigs[i].max()]
    
adv_some_mean_eigs = np.zeros((len(adv_some_eigs),5))
for i in range(len(adv_some_eigs)):
    adv_some_mean_eigs[i] = [adv_some_eigs[i].mean(), adv_some_eigs[i].shape[0], np.median(adv_some_eigs[i]), adv_some_eigs[i].min(), adv_some_eigs[i].max()]

In [ ]:
plt.hist([some_mean_eigs[:,0], adv_some_mean_eigs[:,0]], range=[0, 3.5], bins='auto', color=COLORS, label=PLT_LABELS, edgecolor=EDGE_COLOR)
plt.legend()
plt.title('Distribution of Mean Eigenvalues of Spectrum')

In [ ]:
plt.hist([some_mean_eigs[:,2], adv_some_mean_eigs[:,2]], bins='auto', color=COLORS, label=PLT_LABELS)
plt.legend()
plt.title('Distribution of Median Eigenvalues of Spectrum')

In [ ]:
plt.hist([some_mean_eigs[:,1], adv_some_mean_eigs[:,1]], range=[0, 300], bins='auto', color=COLORS, label=PLT_LABELS, edgecolor=EDGE_COLOR)
plt.legend()
plt.title('Distribution of Spectrum Size')

In [ ]:
plt.hist([some_mean_eigs[:,3], adv_some_mean_eigs[:,3]], range=[-2e-15, 0], bins='auto', color=COLORS, label=PLT_LABELS)
plt.legend()
plt.title('Distribution of Minimum Eigenvalue of Spectrum')

In [ ]:
plt.hist([some_mean_eigs[:,4], adv_some_mean_eigs[:,4]], range=[0,50], bins='auto', color=COLORS, label=PLT_LABELS, edgecolor=EDGE_COLOR)
plt.legend()
plt.title('Distribution of Maximum Eigenvalue of Spectrum')

In [ ]:
options = {
    'node_color': COLORS[0],
    'node_size': 2,
    'width': 2,
    'with_labels':False}
nx.draw_spring(some_gois[2], **options)

In [ ]:
options = {
    'node_color': COLORS[1],
    'node_size': 2,
    'width': 2,
    'with_labels':False}
nx.draw_spring(adv_some_gois[2], **options)

In [ ]:
some_num = 10
some_gois2 = []
for i in range(len(sample_graphs)):
    print(i)
    sgik = list(sample_graphs[i].keys())
    a = [sample_graphs[i][k] for k in sgik[:some_num]]
    some_gois2.append(nx.compose_all(a))
    
adv_some_gois2 = []
for i in range(len(adv_sample_graphs)):
    print(i)
    sgik = list(adv_sample_graphs[i].keys())
    a = [adv_sample_graphs[i][k] for k in sgik[:some_num]]
    adv_some_gois2.append(nx.compose_all(a))
    
some_eigs2 = []
for i in range(len(some_gois)):
    print('normal ', i)
    some_eigs2.append(nx.linalg.laplacian_spectrum(some_gois2[i]))

adv_some_eigs2 = []
for i in range(len(adv_some_gois)):
    print('adv ', i)
    adv_some_eigs2.append(nx.linalg.laplacian_spectrum(adv_some_gois2[i]))

In [ ]:
some_mean_eigs2 = np.zeros((len(some_eigs2),5))
for i in range(len(some_eigs)):
    some_mean_eigs2[i] = [some_eigs2[i].mean(), some_eigs2[i].shape[0], np.median(some_eigs2[i]), some_eigs2[i].min(), some_eigs2[i].max()]
    
adv_some_mean_eigs2 = np.zeros((len(adv_some_eigs2),5))
for i in range(len(adv_some_eigs2)):
    adv_some_mean_eigs2[i] = [adv_some_eigs2[i].mean(), adv_some_eigs2[i].shape[0], np.median(adv_some_eigs2[i]), adv_some_eigs2[i].min(), adv_some_eigs2[i].max()]

In [ ]:
plt.hist([some_mean_eigs2[:,0], adv_some_mean_eigs2[:,0]], range=[2, 8], bins='auto', color=COLORS, label=PLT_LABELS)
plt.legend()
plt.title('Distribution of Mean Eigenvalues of Spectrum')

In [ ]:
plt.hist([some_mean_eigs2[:,2], adv_some_mean_eigs2[:,2]], range=[0.5, 5], bins='auto', color=COLORS, label=PLT_LABELS)
plt.legend()
plt.title('Distribution of Median Eigenvalues of Spectrum')

In [ ]:
plt.hist([some_mean_eigs2[:,1], adv_some_mean_eigs2[:,1]], bins='auto', color=COLORS, label=PLT_LABELS, edgecolor=EDGE_COLOR)
plt.legend()
plt.title('Distribution of Spectrum Size')

In [ ]:
plt.hist([some_mean_eigs2[:,3], adv_some_mean_eigs2[:,3]], bins='auto', color=COLORS, label=PLT_LABELS, edgecolor=EDGE_COLOR)
plt.legend()
plt.title('Distribution of Minimum Eigenvalue of Spectrum')

In [ ]:
plt.hist([some_mean_eigs2[:,4], adv_some_mean_eigs2[:,4]], range=[40, 300], bins='auto', color=COLORS, label=PLT_LABELS)
plt.legend()
plt.title('Distribution of Maximum Eigenvalue of Spectrum')

In [ ]:
plt.imshow(adversaries[5]['adversary'].reshape(28,28))

In [ ]:
options = {
    'node_color': COLORS[0],
    'node_size': 2,
    'width': 2,
    'with_labels':False}
nx.draw_spring(some_gois2[2], **options)

In [ ]:
options = {
    'node_color': COLORS[1],
    'node_size': 2,
    'width': 2,
    'with_labels':False}
nx.draw_spring(adv_some_gois2[2], **options)

In [ ]:
weiners = np.zeros(len(all_gois))
for i in range(len(all_gois)):
    print('normal ', i)
    weiners[i] = nx.algorithms.wiener.wiener_index(all_gois[i], weight='weight')

adv_weiners = np.zeros(len(adv_all_gois))
for i in range(len(adv_all_gois)):
    print('adv ', i)
    adv_weiners[i] = nx.algorithms.wiener.wiener_index(adv_all_gois[i], weight='weight')

In [ ]:
plt.hist([weiners, adv_weiners], bins='auto')

In [ ]:
print(res_df['loss'].max())
res_df.iloc[np.where(weiners == .weiners.max())]

In [ ]:
gk = GraphKernel(kernel={"name": "multiscale_laplacian",
                             "which": "fast",
                             "L": 1,
                             "P": 10,
                             "N": 10})
# gk = grakel.RandomWalkLabeled(method_type='fast', verbose=False)

In [ ]:
for i in range(len(all_gois)):
    l = list(all_gois[i].nodes())
    for j in range(len(l)):
        all_gois[i].node[l[j]]['label'] = l[j]

In [ ]:
for i in range(len(adv_all_gois)):
    l = list(adv_all_gois[i].nodes())
    for j in range(len(l)):
        adv_all_gois[i].node[l[j]]['label'] = l[j]

In [ ]:
G_adv = grakel.graph_from_networkx(adv_all_gois, node_labels_tag='label', edge_weight_tag='weight')
G = grakel.graph_from_networkx(all_gois, node_labels_tag='label', edge_weight_tag='weight')

In [ ]:
K_train = gk.fit_transform(G)